In [1]:
s = 'ABCD'
s2 = '-asdfABCD'
s3 = '#'
s4 = 'abcd-'
s5 = 'ABD-'

In [2]:
import re
from icecream import ic

In [3]:
print(re.match('[A-Z]*[a-z]+|-[a-z][A-Z]*-*', s2))

<re.Match object; span=(0, 2), match='-a'>


In [4]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[A-Z]*[a-z]*|-[a-z][A-Z]*-*', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(0, 3), match='ABD'>


In [5]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('^[A-Z]+|^-|^[a-z]', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 3), match='ABD'>


In [6]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[^#]', elem))

<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='A'>


In [7]:
# comment for recommit

In [8]:
import numpy as np

In [9]:
def read_alignment(filename: str = 'test1.txt') -> list:
    '''
    DESCR
    -----
    Read in sequence alignment(s). 
    Entire file is line-by-line into a list object and stripped of whitespace.
    List will have representation of some row-wise separator in case of multiple alignments.
    
    ARGS
    -----
    filename: name of file
    
    RETURNS
    --------
    list of alignments separated by any separator (if applicable).
    '''
    with open(filename, 'r') as file:
        return [elem.strip() for elem in file.readlines()]
    
    '''
    with open(filename, 'r') as file:
        return [[*elem.strip()] for elem in file.readlines()]
    '''


ic(read_alignment('test1.txt'));
alignments = read_alignment()

def num_alignments(alignments: list) -> int:
    '''
    DESCR
    ------
    Checks to see if how many alignments are in an iterable of alignments by counting
    the number of separators.
    '''
    counter = 0
    for seq in alignments:
        if not re.match('^[A-Z]+|^-|^[a-z]+', seq):
            counter += 1
        else: pass;
    
    # The number of alignments will be the number of separators + 1
    return (counter + 1)

ic(num_alignments(alignments));

ic| read_alignment('test1.txt'): ['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT']
ic| num_alignments(alignments): 2


In [10]:
def columnarize(alignments: list) -> list:
    '''
    This doesnt work for a single alignment; need to rework
    '''
    nested_list = list()
    temp_list = list()

    for i in range(len(alignments)):
        # If element is part of an alignment
        if re.match('^[A-Z]+|^-|^[a-z]+', alignments[i]):
            # add to a temporary alignment
            temp_list.append([alignments[i]])
        # If element is a separator
        else:
            # add full alignment to a list of alignment(s)
            nested_list.append(temp_list)
            # reset temp alignment in case of more alignments
            temp_list = list()
        
    nested_list.append(temp_list)
    return nested_list

ic(columnarize(alignments))
alignments = columnarize(alignments)


NUCLEOTIDES = {'G', 'C', 'A', 'T', '-'}

ic| columnarize(alignments): [[['AAA'], ['ACC'], ['ACG'], ['ACT']], [['AAA-'], ['A-CC'], ['ACG-'], ['A-CT']]]


In [15]:


def test(nested_list, num_alignments):
    temp = ''
    # Loop for each alignment(s)
    for i in range(num_alignments):
        # Loop 
        for i in range(len(nested_list[i])):
            print(nested_list[0][i][0])
            temp += nested_list[0][i][0]
        
test(alignments, num_alignments = num_alignments)

A
A
A
A


In [169]:
{k:0 for k in NUCLEOTIDES}

{'G': 0, 'A': 0, 'C': 0, '-': 0, 'T': 0}

In [97]:
temp

'AAAA'

In [101]:
from collections import Counter
c = Counter('asdf')
c

Counter({'a': 1, 's': 1, 'd': 1, 'f': 1})

In [104]:
Counter('AGC')

Counter({'A': 1, 'G': 1, 'C': 1})

In [ ]:
'AAA', 'ACC', 'ACG', 'ACT'

In [105]:
a = Counter('AAA')
print(a)
b = Counter('ACC')
print(b)
c = Counter('ACG')
print(c)
d = Counter('ACT')
print(d)

Counter({'A': 3})
Counter({'C': 2, 'A': 1})
Counter({'A': 1, 'C': 1, 'G': 1})
Counter({'A': 1, 'C': 1, 'T': 1})


In [81]:
np.array(nested_list, dtype = 'object').reshape(8,1)

array([list(['A', 'C', 'C'])], dtype=object)

In [58]:
np.array(nested_list, dtype = 'object').T

array([[list(['A', 'A', 'A']), list(['A', 'A', 'A', '-'])],
       [list(['A', 'C', 'C']), list(['A', '-', 'C', 'C'])],
       [list(['A', 'C', 'G']), list(['A', 'C', 'G', '-'])],
       [list(['A', 'C', 'T']), list(['A', '-', 'C', 'T'])]], dtype=object)

In [18]:
for i in range(len(seqs)):
    print(re.match('^[A-Z]+|^-|^[a-z]', seqs[i]))

<re.Match object; span=(0, 3), match='AAA'>
<re.Match object; span=(0, 3), match='ACC'>
<re.Match object; span=(0, 3), match='ACG'>
<re.Match object; span=(0, 3), match='ACT'>
None
<re.Match object; span=(0, 3), match='AAA'>
<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 3), match='ACG'>
<re.Match object; span=(0, 1), match='A'>


In [32]:
with open('test1.txt', 'r') as file:
    a = np.array([elem.strip() for elem in file.readlines()], dtype = 'object')
    
a

array(['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT'],
      dtype=object)

In [37]:
def array_regex(i):
    return np.array([True if re.match('[^#]', i) else False])

ar = np.vectorize(array_regex)

c = ar(a)
c

array([ True,  True,  True,  True, False,  True,  True,  True,  True])

In [35]:
def array_regex(i):
    b = np.array([True if re.match('[^#]', i) else False])
    return np.where(True, b)

ar = np.vectorize(array_regex)

c = ar(a)

ValueError: either both or neither of x and y should be given